In [5]:
import pandas as pd
import requests
import regex as re
import csv
from bs4 import BeautifulSoup

In [2]:
# source data; set before running

# use Access query "qry_Course-Schedule-RM-withTIES"; set dates within query to desired range (usually 3 years) (takes a while to run)
course_schedule = r"C:\Work\groupchairsched.xlsx"


# use Access query "qry_IP_SignedOff" for IP info (takes a while to run, be patient; remember to update date field since it's hidden)
# automatically gets catalog for course info (description, title, etc)
instructor_IP = r"C:\Work\groupchairip.xlsx"


# continuing lecturers and their appointment %
instructor_appointments = r""


# areas of emphasis and certificate info
emphasis_certs = r"C:\Work\Course Areas of Emphasis and Certificates.xlsx"


# campus catalog course data (no action necessary, runs in cells below)
guide_urls = ["""http://guide.berkeley.edu/courses/ugba/""",
       """http://guide.berkeley.edu/courses/ewmba/""",
       """http://guide.berkeley.edu/courses/xmba/""",
       """http://guide.berkeley.edu/courses/mba/""",
       """http://guide.berkeley.edu/courses/phdba/""",
       """http://guide.berkeley.edu/courses/mfe/"""]


# budget estimate report from curricular planning reports
# using 'data dump' tab
budget_report = r"C:\Work\CPBudgetEstimate_06022022_225820.xlsx"


In [3]:
# read in and clean up schedule data

schedule_df = ""


In [4]:
# get guide data
columns = ['Course Number', 'Course Title', 'Description', 'Units']
df = pd.DataFrame(columns=columns)

def get_courses(url, df):
    header = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/64.0.3282.186 Safari/537.36'}
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.content, 'html.parser')
    courseblocks = soup.find_all("div", {"class": "courseblock"})
    for courseblock in courseblocks:
        courseblock.a.decompose() # this deletes any "a" tags in the course. Doing this to remove the bottom "read more" link
        course = courseblock.find("span", {"class": "code"}).text
        title = courseblock.find("span", {"class": "title"}).text
        units = courseblock.find("span", {"class": "hours"}).text
        desc = courseblock.find('p', {'class': 'courseblockdesc'}).text
        desc = re.sub(r'\bTerms offered\b(.)*', '', desc)
        desc = re.sub('\\n*', '', desc)
        course_append = {'Course Number': course, 'Course Title': title, 'Description': desc, 'Units': units}
        df = df.append(course_append, ignore_index=True)
    return df


for url in guide_urls:
        df = get_courses(url, df)

print(df)


    Course Number                                       Course Title  \
0         UGBA C5                   Introduction to Entrepreneurship   
1         UGBA 10                             Principles of Business   
2        UGBA C12                           The Berkeley Changemaker   
3         UGBA 13                 Berkeley Changemaker: Human Health   
4         UGBA 24                                  Freshman Seminars   
..            ...                                                ...   
528      MFE 230X                             High Frequency Finance   
529      MFE 230Y         Ethics and Regulation in Financial Markets   
530     MFE 230ZA                   Deep Learning and Applications I   
531     MFE 230ZB                  Deep Learning and Applications II   
532       MFE 293  Individually Supervised Study for Graduate Stu...   

                                           Description         Units  
0    This course offers students a taste of what it...       2 U

In [ ]:
# create budget sheet per group

budget_df = ""

In [ ]:
# this is example code of how to beautiful soup read pages
# delete once you have the guide data working

# this file should have the urls of each faculty member's Haas website
INPUT_FACULTY_FILE = r"C:\Work\haas_faculty_urls.txt"
OUTPUT_FILE = r"C:\Work\fac_education_rev.xlsx"

columns = ['Berkeley_ID', 'Employee_Name', 'Institution', 'Location', 'Major', 'Degree']

# df = pd.read_csv(INPUT_FACULTY_FILE, header=None, names=["url"])
df = pd.DataFrame(columns=columns)


# for any given haas faculty url, goes to the site then looks for the education field and copies the contents.
def get_faculty_education(url, df):
    header = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                            'Chrome/64.0.3282.186 Safari/537.36'}
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        name = soup.find("h1", itemprop="name headline").text
    except AttributeError:
        name = ""
    try:
        div_contents = soup.find('h2', text='Education').findParent()
        lis = div_contents.findAll("li")
        p_text = div_contents.findAll("p")
        if lis:
            for li in lis:
                new_row = {'Employee_Name': name, "Education": li.text, "URL": url}
                df = df.append(new_row, ignore_index=True)
        else:
            for p in p_text:
                new_row = {'Employee_Name': name, "Education": p.text, "URL": url}
                df = df.append(new_row, ignore_index=True)
    except AttributeError:
        pass

    print("%s processed." % name)
    return df


def stripper(text):
    fields = text.split(",")
    return_series = pd.Series(fields)
    if return_series.size == 3:
        return return_series
    else:
        return text, "", ""


# goes through file and creates a row in dataframe for each education record
with open(INPUT_FACULTY_FILE) as csvfile:
    file = csv.reader(csvfile)
    for row in file:
        url = row[0]
        df = get_faculty_education(url, df)

df[['Degree', 'Major', 'Institution']] = df.apply(lambda x: stripper(x["Education"]), axis=1)

strip_columns = ['Degree', 'Major', 'Institution']
for column in strip_columns:
    df[column] = df[column].str.strip()

print(df)
df.to_excel(OUTPUT_FILE, columns=columns, index=False)